

## California Housing Data

El conjunto de datos "California Housing" contiene información sobre el precio medio de las casas en varios distritos de California, así como información relacionada con las características socioeconómicas y geográficas de cada distrito.

Atributos:
<ul>
<li>Latitude: Latitud geográfica del centro del distrito (numérico).</li>
<li>Longitude: Longitud geográfica del centro del distrito (numérico).</li>
<li>House_median_age: Mediana de la edad de las viviendas en el distrito (numérico).</li>
<li>Total_rooms: Total de habitaciones por vivienda en el distrito (numérico).</li>
<li>Total_bedrooms: Total de dormitorios por vivienda en el distrito (numérico).</li>
<li>Population: Población total en el distrito (numérico).</li>
<li>Households: Número promedio de personas por vivienda en el distrito (numérico).</li>
<li>Median_income: Mediana del ingreso de los hogares en el distrito (numérico).</li>
<li>Median_house_value: clase objetivo. Mediana del valor de las viviendas en el distrito (numérico).</li>
<li>Ocean_proximity: localización de la vivienda. (categórico)</li>
</ul>

Se puede obtener en https://www.kaggle.com/datasets/camnugent/california-housing-prices

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import RootMeanSquaredError

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/housing.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Cargar el conjunto de datos
housing=pd.read_csv(path,sep=",", header=0)

In [4]:
# Identificar filas con valores nulos
rows_with_nans = housing[housing.isnull().any(axis=1)]
print(rows_with_nans)

# Reemplazar valores faltantes
housing = housing.dropna()

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
290      -122.16     37.77                47.0       1256.0             NaN   
341      -122.17     37.75                38.0        992.0             NaN   
538      -122.28     37.78                29.0       5154.0             NaN   
563      -122.24     37.75                45.0        891.0             NaN   
696      -122.10     37.69                41.0        746.0             NaN   
...          ...       ...                 ...          ...             ...   
20267    -119.19     34.20                18.0       3620.0             NaN   
20268    -119.18     34.19                19.0       2393.0             NaN   
20372    -118.88     34.17                15.0       4260.0             NaN   
20460    -118.75     34.29                17.0       5512.0             NaN   
20484    -118.72     34.28                17.0       3051.0             NaN   

       population  households  median_income  media

In [5]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
# Ver los nombres de las columnas
print(housing.columns)

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')


In [7]:
# Ver las categorías únicas en la columna 'ocean_proximity'
print(housing['ocean_proximity'].unique())

['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']


In [8]:
# Crear una instancia de LabelEncoder
label_encoder = LabelEncoder()

# Ajustar y transformar la columna 'ocean_proximity'
housing['ocean_proximity_encoded'] = label_encoder.fit_transform(housing['ocean_proximity'])

# Mostrar el DataFrame resultante con la nueva columna codificada
print(housing[['ocean_proximity', 'ocean_proximity_encoded']].head())

  ocean_proximity  ocean_proximity_encoded
0        NEAR BAY                        3
1        NEAR BAY                        3
2        NEAR BAY                        3
3        NEAR BAY                        3
4        NEAR BAY                        3


In [9]:
# Eliminar la columna antigua 'ocean_proximity'
housing.drop('ocean_proximity', axis=1, inplace=True)

In [10]:
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_encoded
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,1
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,1
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,1
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,1


In [11]:
# División de características y etiquetas
y = housing['median_house_value']
X = housing.drop(columns=['median_house_value'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
input_shape=X_train_scaled.shape[1]

In [15]:
# Crear un modelo secuencial
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu'),
    Dense(1)
])

In [16]:
optimizer = Adam(learning_rate=0.001)

In [17]:
# Compilar el modelo
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

In [18]:
# Resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2753 (10.75 KB)
Trainable params: 2753 (10.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=16, validation_split=0.2)

Epoch 1/20
818/818 [==============================] - 3s 3ms/step - loss: 55144816640.0000 - mean_absolute_error: 204989.6562 - val_loss: 53675069440.0000 - val_mean_absolute_error: 201350.0312
Epoch 2/20
818/818 [==============================] - 3s 3ms/step - loss: 47148273664.0000 - mean_absolute_error: 186315.7344 - val_loss: 39978901504.0000 - val_mean_absolute_error: 167922.5938
Epoch 3/20
818/818 [==============================] - 2s 3ms/step - loss: 30217965568.0000 - mean_absolute_error: 140308.1719 - val_loss: 22653442048.0000 - val_mean_absolute_error: 116931.7891
Epoch 4/20
818/818 [==============================] - 2s 2ms/step - loss: 16871563264.0000 - mean_absolute_error: 97796.3750 - val_loss: 14208285696.0000 - val_mean_absolute_error: 89014.0312
Epoch 5/20
818/818 [==============================] - 2s 2ms/step - loss: 12100653056.0000 - mean_absolute_error: 81384.9844 - val_loss: 11685771264.0000 - val_mean_absolute_error: 79864.2422
Epoch 6/20
818/818 [==============

In [20]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {test_mae}')


128/128 [==============================] - 0s 2ms/step - loss: 419110894698496.0000 - mean_absolute_error: 16590821.0000
Test Mean Absolute Error: 16590821.0


## Parámetros avanzados


In [21]:
# Definir la arquitectura del modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],),
          kernel_initializer=HeNormal(), kernel_regularizer=l1(0.01)),
    Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01)), # se puede utilizar "he_normal"
    Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01)),
    Dense(1)  # Una única salida para la regresión
])

In [22]:
# Definir el optimizador con una tasa de aprendizaje personalizada
optimizer =  RMSprop(learning_rate=0.001)

In [23]:
# Compilar el modelo
model.compile(optimizer=optimizer, loss='mae', metrics=['mse'])


In [24]:
# Resumen del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               1280      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11649 (45.50 KB)
Trainable params: 11649 (45.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Definir callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

callbacks = [early_stopping, model_checkpoint]

In [26]:
# Entrenar el modelo con callbacks
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=callbacks)


Epoch 1/100
409/409 [==============================] - 2s 3ms/step - loss: 124366.5547 - mse: 29684994048.0000 - val_loss: 113046.8672 - val_mse: 25719031808.0000
Epoch 2/100
 90/409 [=====>........................] - ETA: 0s - loss: 113373.8203 - mse: 26145828864.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


409/409 [==============================] - 1s 2ms/step - loss: 103767.0312 - mse: 21740134400.0000 - val_loss: 91066.4688 - val_mse: 15787097088.0000
Epoch 3/100
409/409 [==============================] - 1s 2ms/step - loss: 83628.7578 - mse: 12587137024.0000 - val_loss: 82106.8203 - val_mse: 11506162688.0000
Epoch 4/100
409/409 [==============================] - 1s 2ms/step - loss: 79789.2109 - mse: 10953773056.0000 - val_loss: 81274.8125 - val_mse: 11542557696.0000
Epoch 5/100
409/409 [==============================] - 1s 2ms/step - loss: 77615.4531 - mse: 10294171648.0000 - val_loss: 79066.8828 - val_mse: 10861317120.0000
Epoch 6/100
409/409 [==============================] - 1s 2ms/step - loss: 75046.3203 - mse: 9521451008.0000 - val_loss: 73699.6875 - val_mse: 9073406976.0000
Epoch 7/100
409/409 [==============================] - 1s 2ms/step - loss: 73042.8906 - mse: 9030173696.0000 - val_loss: 70987.9141 - val_mse: 8746200064.0000
Epoch 8/100
409/409 [============================

In [27]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {test_mae}')


128/128 [==============================] - 0s 1ms/step - loss: 51304.6602 - mse: 4922137600.0000
Test Mean Absolute Error: 4922137600.0


## Con una sola capa

In [28]:
model_simple = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1)
])

In [29]:
# De pérdida, Huber. Es una combinación entre la pérdida cuadrática y la pérdida absoluta, utilizada en modelos de regresión para ser menos sensible a valores atípicos. Se comporta como una pérdida cuadrática para errores pequeños y como una pérdida absoluta para errores grandes.
loss = Huber(delta=1.0)
metrics = [RootMeanSquaredError()]

In [30]:
model_simple.compile(optimizer='adam', loss=loss, metrics=metrics)

In [31]:
history_simple = model_simple.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
409/409 [==============================] - 2s 4ms/step - loss: 184288.3750 - root_mean_squared_error: 218020.2812 - val_loss: 145505.4531 - val_root_mean_squared_error: 184848.6406
Epoch 2/50
409/409 [==============================] - 1s 3ms/step - loss: 123569.4375 - root_mean_squared_error: 167215.5625 - val_loss: 116801.6797 - val_root_mean_squared_error: 163918.5938
Epoch 3/50
409/409 [==============================] - 1s 3ms/step - loss: 115662.3438 - root_mean_squared_error: 162856.5156 - val_loss: 115741.6016 - val_root_mean_squared_error: 163737.0000
Epoch 4/50
409/409 [==============================] - 1s 2ms/step - loss: 114491.6328 - root_mean_squared_error: 161468.2188 - val_loss: 114334.3438 - val_root_mean_squared_error: 161581.7031
Epoch 5/50
409/409 [==============================] - 1s 2ms/step - loss: 112833.1641 - root_mean_squared_error: 159008.0156 - val_loss: 112419.2500 - val_root_mean_squared_error: 158565.5938
Epoch 6/50
409/409 [====================

In [34]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = model_simple.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {test_mae}')

128/128 [==============================] - 0s 1ms/step - loss: 10805102592.0000 - mae: 81738.8359
Test Mean Absolute Error: 81738.8359375


In [35]:
# Otra estructura. kernel_initializer='he_normal' asigna pesos iniciales en una red neuronal usando una distribución normal con una desviación estándar escalada según el tamaño de la capa previa, optimizando el rendimiento en capas con activaciones ReLU.
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_initializer='he_normal'),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.2),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    Dense(1)
])

In [36]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [37]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100
409/409 [==============================] - 3s 4ms/step - loss: 29352247296.0000 - mae: 126945.0156 - val_loss: 23790845952.0000 - val_mae: 109830.3047
Epoch 2/100
409/409 [==============================] - 1s 2ms/step - loss: 19193243648.0000 - mae: 99643.9766 - val_loss: 13185857536.0000 - val_mae: 85830.0391
Epoch 3/100
409/409 [==============================] - 1s 2ms/step - loss: 12660459520.0000 - mae: 85978.6719 - val_loss: 11365786624.0000 - val_mae: 82063.4453
Epoch 4/100
409/409 [==============================] - 1s 2ms/step - loss: 11712276480.0000 - mae: 84249.1172 - val_loss: 10579338240.0000 - val_mae: 80132.6562
Epoch 5/100
409/409 [==============================] - 1s 2ms/step - loss: 11105456128.0000 - mae: 82235.7109 - val_loss: 10131670016.0000 - val_mae: 78982.5547
Epoch 6/100
409/409 [==============================] - 1s 2ms/step - loss: 10905103360.0000 - mae: 81623.7344 - val_loss: 9706084352.0000 - val_mae: 77200.4375
Epoch 7/100
409/409 [============

In [40]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {test_mae}')

128/128 [==============================] - 0s 3ms/step - loss: 4599436800.0000 - mae: 48516.6797
Test Mean Absolute Error: 48516.6796875


## Prueba con CNN
Una red neuronal convolucional (CNN) es un tipo de red neuronal diseñada para procesar datos con una estructura de grid (como imágenes), usando capas convolucionales para detectar patrones locales y capas de pooling para reducir dimensionalidad y mantener características importantes. Son ampliamente usadas en tareas de visión por computadora y reconocimiento de patrones.


In [41]:
# Reshape los datos para Conv1D
X_train_cnn = np.expand_dims(X_train, axis=-1)
X_test_cnn = np.expand_dims(X_test, axis=-1)

In [42]:
model_cnn = Sequential([
    # Aplica 32 filtros 1D con tamaño de kernel 2 y activación ReLU a la entrada, extrayendo características locales.
    Conv1D(32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    # Reduce la dimensión espacial de las características extraídas tomando el valor máximo en ventanas de tamaño 2, ayudando a la reducción de dimensionalidad y control de sobreajuste.
    MaxPooling1D(pool_size=2),
    # Convierte las características 1D en un vector plano, preparándolas para ser ingresadas a capas densas (fully connected) posteriores.
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

In [43]:
model_cnn.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [44]:
history_cnn = model_cnn.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
409/409 [==============================] - 3s 4ms/step - loss: 34870140928.0000 - mae: 144723.0312 - val_loss: 26507577344.0000 - val_mae: 118102.7344
Epoch 2/50
409/409 [==============================] - 1s 3ms/step - loss: 25193426944.0000 - mae: 114155.6094 - val_loss: 24396105728.0000 - val_mae: 113116.0781
Epoch 3/50
409/409 [==============================] - 1s 2ms/step - loss: 22378979328.0000 - mae: 106950.0000 - val_loss: 20815439872.0000 - val_mae: 103376.2734
Epoch 4/50
409/409 [==============================] - 1s 2ms/step - loss: 18345750528.0000 - mae: 97312.9375 - val_loss: 16445499392.0000 - val_mae: 93491.6875
Epoch 5/50
409/409 [==============================] - 1s 2ms/step - loss: 14536492032.0000 - mae: 89237.5547 - val_loss: 13549419520.0000 - val_mae: 87496.3203
Epoch 6/50
409/409 [==============================] - 1s 2ms/step - loss: 12730976256.0000 - mae: 86400.8047 - val_loss: 12497166336.0000 - val_mae: 87122.9141
Epoch 7/50
409/409 [==============

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = model_cnn.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {test_mae}')